# <center> Regressão Linear Multivariada </center>
## <center> Modelo Preditivo </center> 



### <center> Projeção para Preço de Imoveis Residenciais na Cidade de São Paulo </center>

<center> <img src="https://s3.static.brasilescola.uol.com.br/be/2024/01/sao-paulo-470-anos.jpg" width=1000 height=350 /> </center>

São Paulo, centro financeiro do Brasil, está dentre as cidades mais populosas do mundo, com diversas instituições culturais, um divercificado cenário gastonomico e uma rica tradição arquitetônica. 

Há prédios e edificios simbólicos na cidade, e quase que todo ano, se constroi um novo predio ou edificio, uma nova casa, uma nova residencia, e com isso, se faz extremamente necessario saber exatamente o quanto vale o imovel residencial na cidade de são paulo e quais são os componentes principais que definem esse preço, para que deste modo, possa estabelecer um valor correto no imovel.

Com o proposito de entender a relação entre o preço dos imoveis residenciais a venda na ciadade de São Paulo, e seus fatores ponderantes na composição do valor do imovel, e em trazer previsibilidade ao setor imobiliario residencial na cidade de são paulo, foi desenvolvivo um projeto com ciencia de dados, para um modelo preditivo baseado em regressão linear multivariada, e para tal, foram utilizados dados historicos de imoveis anunciados no mes de fevereiro no ano de 2024 no portal vivareal.com.br de anuncios de venda de imoveis em todo o Brasil, e forá utilizado somente dados de imoveis residenciais na cidade de são paulo, entre casa, que também estão inseridos os sobrados, as kitnets ou conjugados, apartamentos e coberturas, e todos estes dados foram extraidos com o metodo de web scraping por meio da linguegem python, onde foram devidamente estruturados em variaveis.  

## 1º Parte - Web Scraping (Extração dos Dados)

Este projeto tem por obejtivo servir como fonte de pesquisa e estudo, sem qualquer fim financeiro ou comercial, somente o compartilhamento de conhecimento, aplicado a um problema que pode vir a ser enfrentando no dia a dia de um profissional da area de dados.

Esta é uma das partes principais deste projeto, esta primeira etapa da modelagem preditiva visa captar os dados, a materia prima do modelo, buscando extrair todas as variaveis relevantes e que sejam possiveis de serem extraidas.

In [47]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time

In [48]:
# Quqantidade de paginas a serem raspadas
paginas = 277

# Endereço de site para webscraping
url = 'https://www.vivareal.com.br/feirao/festival-da-mudanca/?pagina={npagina}#com=sale&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,,,,city,BR%3ESao%20Paulo%3ENULL%3ESao%20Paulo,,,&tipos=apartamento_residencial,casa_residencial,condominio_residencial,cobertura_residencial,kitnet_residencial,sobrado_residencial,'

# Contador de pagina
cont= 0

# Lista com os dados da raspagem
fichas =[]

# ID fake do browser
headers={
'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15"}

# Gerando um Looping para capturar a resposta da requisição das paginas
for pagina in range(1,paginas+1):
    # Contador
    cont += 1
    # Visualizar a contagem e a url da pagina em raspagem
    print(f"Pag Nº {cont} Estou aqui: " + url.format(npagina=pagina))

    # Gerando um objeto para a sessão de requisição
    session = requests.Session()

    # Autenticando ID Fake
    session.headers.update(headers)
    time.sleep(20.0)
    # Gerando uma Requisição HTTP para uma resposta em HTML
    response = session.get(url.format(npagina=pagina), timeout=40)

    # Visualizando o Status da Requisição
    print(response.status_code)
    # 200 Sucesso, 403 Requisição negada, 404 enderço url não encontrado

    # Reestruturando para facil manipulação e visualização
    site_soup = BeautifulSoup(response.content, 'html.parser')

    # Raspando todos os cartões de anuncio da pagina
    site_cartao = site_soup.find_all('div', class_= "js-card-selector")

    # Gerando um Looping para raspar os dados contidos em cada cartão de anuncio
    for cartao_full  in site_cartao:

        # Criando uma lista com todos os dados raspados dos cartões de anuncio
        tabela = {}

        # Numero serial de identificação do anuncio
        tabela['Serial_ID'] = int(str(cartao_full.parent.get('id')))

    # Craindo coluna Tipo de Imovel
        try:
            tabela['Tipo'] = str(cartao_full.find('h2', attrs={'class':'property-card__header'}).text.strip().split()[0:4])
        except:
            tabela['Tipo'] = 'Sem'

        # Gerando coluna metro quadrado do imovel
        try:
            tabela['Area_M²'] = str(cartao_full.find('ul', attrs={'class':"property-card__details"}).text.strip().split()[0])
        except:
            tabela['Area_M²'] = 'Sem'

        # Coluna Endereço
        try:
            tabela['End'] = str(cartao_full.find('span', class_="property-card__address").text)
        except:
            tabela['End'] = 'Sem'

        # Coluna Nº de Quartos
        try:
            tabela['N_Quarto'] = str(cartao_full.find('li', class_="property-card__detail-item property-card__detail-room js-property-detail-rooms").text.strip().split()[0])
        except:
            tabela['N_Quarto'] = 'Sem'

        # Coluna Nº de banheiros
        try:
            tabela['N_Banheiro'] = str(cartao_full.find('li',class_="property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom").text.strip().split()[0])
        except:
            tabela['N_Banheiro'] = 'Sem'

        # Coluna Nº de vagas
        try:
            tabela['N_Vagas'] = str(cartao_full.find('li', class_="property-card__detail-item property-card__detail-garage js-property-detail-garages").text.strip().split()[0])
        except:
            tabela['N_Vagas'] = 'Sem'

        # Coluna Lazer
        try:
            tabela['Lazer'] = str(cartao_full.find('ul', class_="property-card__amenities").text.strip().split())
        except:
            tabela['Lazer'] = 'Sem'

        # Coluna Valor Condominio
        try:
            tabela['R$_Cond'] = cartao_full.find('div', class_="property-card__price-details--condo").text.strip().split()[-1]
        except:
            tabela['R$_Cond'] = 'Sem'

        # Coluna Valor do Imovel
        try:
            tabela['R$_Imovel'] = cartao_full.find('div', class_="property-card__price js-property-card-prices js-property-card__price-small").text.replace('R$', '').strip().replace('.', '')
        except:
            tabela['R$_Imovel'] = 'Sem'


        # Adiciona apenas elementos unicos (Evitar anuncios em duplicidade)
        if tabela not in fichas:
            fichas.append(tabela)
        time.sleep(0.20)

# Convertendo a lista com os dados em dataframe
df_imov_br = pd.DataFrame(fichas, columns=tabela.keys())

# Salvando o dataframe em um formato de arquivo CSV
df_imov_br.to_csv('DB_Imoveis_sp_Bruto_parte_4.csv', index_label=False)

Pag Nº 1 Estou aqui: https://www.vivareal.com.br/feirao/festival-da-mudanca/?pagina=1#com=sale&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,,,,city,BR%3ESao%20Paulo%3ENULL%3ESao%20Paulo,,,&tipos=apartamento_residencial,casa_residencial,condominio_residencial,cobertura_residencial,kitnet_residencial,sobrado_residencial,
200
Pag Nº 2 Estou aqui: https://www.vivareal.com.br/feirao/festival-da-mudanca/?pagina=2#com=sale&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,,,,city,BR%3ESao%20Paulo%3ENULL%3ESao%20Paulo,,,&tipos=apartamento_residencial,casa_residencial,condominio_residencial,cobertura_residencial,kitnet_residencial,sobrado_residencial,
200
Pag Nº 3 Estou aqui: https://www.vivareal.com.br/feirao/festival-da-mudanca/?pagina=3#com=sale&onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Paulo,,,,,city,BR%3ESao%20Paulo%3ENULL%3ESao%20Paulo,,,&tipos=apartamento_residencial,casa_residencial,condominio_residencial,cobertura_residencial,kitnet_residencial,sobrado_residencial,
200
Pag Nº 4 Estou aqui: https://www.viva